In [ ]:
# Embedded Reber grammars were used by Hochreiter and Schmidhuber in their paper about LSTMs. 
# They are artificial grammars that produce strings such as "BPBTSXXVPSEPE." 
# Check out Jenny Orr's nice introduction to this topic. 
# Choose a particular embedded Reber grammar (such as the one represented on Jenny Orr's page), 
# then train an RNN to identify whether a string respects that grammar or not. 
# You will first need to write a function capable of generating a training batch containing about 50% strings 
# that respect the grammar, and 50% that don't.

In [ ]:
# 1: function that generates strings based on a grammar
# The grammar will be represented as a list of possible transitions for each state. 
# A transition specifies the string to output (or a grammar to generate it) and the next state.

default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [ ]:
# generate a few strings based on the default Reber grammar:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end=" ")

In [ ]:
# generate a few strings based on the embedded Reber grammar:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=" ")

In [ ]:
# Function to generate strings that do not respect the grammar
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

In [ ]:
# ...and use it to generate a few string not following grammar:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

In [ ]:
# We cannot feed strings directly to an RNN, so we need to encode them somehow. 
# One option would be to one-hot encode each character. Another option is to use embeddings. 
# Let's go for the second option (but since there are just a handful of characters, 
# one-hot encoding would probably be a good option as well). 
# For embeddings to work, we need to convert each string into a sequence of character IDs. 
# Let's write a function for that, using each character's index in the string of possible characters "BEPSTVX":


def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [POSSIBLE_CHARS.index(c) for c in s]

string_to_ids("BTTTXXVVETE")

In [ ]:
# add a funciton to generate a dataset based on embeddings
def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar))
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar))
                   for _ in range(size - size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [ ]:
# ... use it to generate the dataset
np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

In [1]:
# List first sequence: 
X_train[0]
# .. and classes it belongs to
y_train[0]

In [ ]:
# Create an RNN that identifies the strings that follow the grammar. Such RNN will implement a binary classifier 

np.random.seed(42)
tf.random.set_seed(42)

embedding_size = 5

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])
optimizer = keras.optimizers.SGD(lr=0.02, momentum = 0.95, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
# test our RNN on two tricky strings: the first one is bad while the second one is good. 
# They only differ by the second to last character. If the RNN gets this right, 
# it shows that it managed to notice the pattern that the second letter should always be equal 
# to the second to last letter. That requires a fairly long short-term memory 
# (which is the reason why we used a GRU cell).

test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))